#### Tpot

- TPOT는 예측 모델링 작업을 위한 고품질 기계 학습 모델을 자동으로 발견하기 위해 널리 사용되는 AutoML 라이브러리

- genetic programming으로 머신러닝 파이프라인을 최적화

- 수천 개의 가능한 파이프라인 중에서 가장 적합한 것을 사용하여 규칙적이고 지루한 작업을 자동화하는 데 도움

http://epistasislab.github.io/tpot/

- 사용이 매우 간편하지만 높은 성능을 얻으려면 시간이 많이 걸림

- TPOT은 같은 데이터셋이라도 다른 결과를 낼 수 있음, 최적화 기법에 랜덤성이 있기 때문 >> 이 점을 이용해 해당 데이터셋에 적합한 모델을 찾는데에 유용

In [1]:
!pip install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install category-encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import sys, os, os.path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

from category_encoders.cat_boost import CatBoostEncoder

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')


In [4]:
train.head(3)

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B


#### 전처리
필요없는 칼럼 드랍하고 종속변수 할당해주는 함수 생성

In [5]:
def get_x_y(df):
  if 'class' in df.columns:   #train 데이터셋의 경우, 여기서 X_train, X_test를 만듬
    df_x = df.drop(columns=['id', 'class'])
    df_y = df['class']
    return df_x, df_y
  else:    #test 데이터셋의 경우, 여기서 y_train을 만듬
    df_x = df.drop(columns=['id'])
    return df_x

X_train, y_train = get_x_y(train)
X_test = get_x_y(test)

라벨인코딩

- zfill(): 문자열 앞 0으로 채우기

  - in: "3".zfill(3) >> out: 003

  - in: "s".zfill(4) >> out: 000s

In [6]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
#타겟과 변수에 각각 인코딩할 수 있도록

snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1, 16)]
#SNP_01부터 15까지 f스트링과 for문을 이용하여 snp_col라는 리스트에 할당

snp_data=[]
for col in snp_col:
  snp_data += list(X_train[col].values)
#snp_data라는 리스트에 X_train에 있는 'SNP_01~15'까지의 칼럼값을 넣어줌

y_train = class_le.fit_transform(y_train)
#class만 할당되어 있는 y_train에 라벨인코딩 진행

snp_le.fit(snp_data)
#'SNP_01~15'까지의 칼럼값만 모아둔 리스트에도 라벨인코딩 진행

for col in X_train.columns:
  if col in snp_col:
    X_train[col] = snp_le.transform(X_train[col])
    X_test[col] = snp_le.transform(X_test[col])
#X_train과 X_test에 있는 'SNP_01~15'까지의 칼럼에도 라벨인코딩 진행

X_train['label'] = y_train

X_train = X_train.iloc[:, 3:]
X_test = X_test.iloc[:, 3:]

데이터 분리

In [7]:
X = X_train.drop(columns=['label'])
y = X_train['label']

X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X, y, test_size=0.2, random_state=156)
#여기서의 데이터분리는 tpot모델에 넣기 위함

In [8]:
X_TRAIN.head()

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
231,2,5,1,3,0,0,1,4,4,0,1,1,0,1,0,4
41,2,5,1,2,0,3,1,0,5,0,5,1,4,1,0,0
149,2,5,1,3,0,3,0,0,5,4,5,5,0,0,0,4
100,1,1,5,0,5,0,5,5,0,5,0,5,4,5,2,4
122,2,5,0,0,0,2,5,0,4,0,0,5,0,1,0,4


#### 모델 학습

In [10]:
#사용하고자 하는 평가지표를 sklearn의 make_scorer를 통해 만들어주기
#tpot모델 파라미터에 넣어주면됌

def my_custom_accuracy(y_true, y_pred):
  score = f1_score(y_true, y_pred, average='macro')
  return score

my_custom_scorer = make_scorer(my_custom_accuracy, greater_is_better=True)

- tpot 모델 파라미터

 - generations: default=100, 몇 세대에 걸쳐서 모델을 탐색할건지 결정. tpot 모델이 돌아가는 시간을 결정하는 가장 중요한 파라미터 중 하나. 값이 클수록 높은 성능의 모델을 찾을 확률이 높다

 - population_size: default=100, 매 generation마다 남겨놓을 파이프라인 수를 설정. 값이 클수록 높은 성능의 모델을 찾을 확률이 높다

 - offspring_size: default=None, 매 generation마다 생성할 자손의 수. 디폴트로 설정하면 population_size로 설정

 !! TPOT은 위의 세가지 파라미터로 총 generations * offspring_size + population_size 개의 pipeline을 평가한다.

 - scoring: 파이프라인을 평가할 지표를 설정.

 - max_eval_time_mins: default=5, 하나의 파이프라인을 평가하는데 드는 시간(min)을 설정. 값을 크게 설정할수록 좀 더 복잡한 파이프라인을 평가

In [11]:
#tpot으로 학습

tpot = TPOTClassifier(generations=50, population_size=50, 
                      verbosity=2, random_state=42,
                      scoring=my_custom_scorer, max_eval_time_mins=5)
tpot.fit(X_TRAIN, Y_TRAIN)
print(tpot.score(X_TEST, Y_TEST))
tpot.export('tpot_digits_pipeline_test.py')

Optimization Progress:   0%|          | 0/2550 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9527012987012988

Generation 2 - Current best internal CV score: 0.9672400416920542

Generation 3 - Current best internal CV score: 0.9672400416920542

Generation 4 - Current best internal CV score: 0.9672400416920542

Generation 5 - Current best internal CV score: 0.9672400416920542

Generation 6 - Current best internal CV score: 0.9672400416920542

Generation 7 - Current best internal CV score: 0.9672400416920542

Generation 8 - Current best internal CV score: 0.9672400416920542

Generation 9 - Current best internal CV score: 0.9672400416920542

Generation 10 - Current best internal CV score: 0.9672400416920542

Generation 11 - Current best internal CV score: 0.9672400416920542

Generation 12 - Current best internal CV score: 0.9672400416920542

Generation 13 - Current best internal CV score: 0.9677033492822966

Generation 14 - Current best internal CV score: 0.9677033492822966

Generation 15 - Current best internal CV score: 0.967703

모델 학습이 종료되면 최적의 파이프라인을 py파일로 저장할 수 있음. 

여기서 모델 부분을 가져와 손쉽게 사용할 수 있음

#### 최종 학습, 예측

최적의 파이프라인에서 모델을 가져와 원하는 데이터로 다시 학습시키고 예측하기

- hasattr(object, name): object의 속성(attribute) 존재를 확인

 - 만약 argument로 넘겨준 object에 name의 속성이 존재하면 Ture, 아니면 False를 반

- setattr(object, name, value): object에 존재하는 속성의 값을 바꾸거나, 새로운 속성을 생성하여 값을 부여

In [18]:
# 훈련데이터셋의 평균 CV score: 0.9724355219752537
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB

#다층 인공신경망 모델 Multi-Layer Perceptron
exported_pipeline = MLPClassifier(alpha=0.001, learning_rate_init=0.01)

#random_state 고정시키기
if hasattr(exported_pipeline, 'random_state'):
  setattr(exported_pipeline, 'random_state', 42)

In [19]:
X

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,2,5,1,0,4,2,0,0,5,0,5,1,0,0,0,0
1,2,1,1,2,0,0,1,0,4,0,1,0,4,5,0,0
2,2,5,5,0,4,3,5,0,4,4,1,0,0,0,0,0
3,1,0,5,0,4,0,5,5,0,5,1,5,5,5,0,5
4,2,5,5,3,0,3,0,0,0,0,5,0,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2,1,1,0,4,3,1,0,4,0,5,1,4,0,0,0
258,2,5,0,2,0,0,1,4,4,0,1,1,0,1,0,4
259,1,1,5,0,4,0,1,5,4,4,0,5,5,5,2,5
260,1,0,5,0,4,0,5,5,0,4,1,1,4,5,2,5


In [20]:
exported_pipeline.fit(X, y_train)
#여기서 쓰는 훈련용 데이터는 tpot학습을 위해 분리하기 전의 데이터

results = exported_pipeline.predict(X_test)

In [21]:
submit = pd.read_csv('/content/sample_submission.csv')

In [22]:
#앞서 라벨인코더를 통해 인코딩한 target값을 inverse_transform()를 이용해 다시 A,B,C로 변환
submit['class'] = class_le.inverse_transform(results)

In [23]:
submit.to_csv('gene_submit_AutoML_tpot.csv', index=False)

In [24]:
submit

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,C
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B
